# Preliminaries
(Installations, imports, definitions. Don't read, just run it.)

In [ ]:
!pip install z3-solver

In [ ]:
import sympy as sp
from IPython.display import display


x, y, u = sp.symbols('x y u')
a, b, d, e, p, q  = sp.symbols('a b d e p q')


def lie(field, F):
    vars = sorted(list(filter(lambda s: 'x' in s.name or 'y' in s.name, F.free_symbols)),
                  key=lambda s: s.name)
    xs, ys = vars[:len(vars)//2], vars[len(vars)//2:]
    vars = [val for pair in zip(xs, ys) for val in pair]
    print(vars)
    return (sp.Matrix([F]).jacobian(vars) @ field)

def fc_from_VGH(V, G, H, u=sp.Matrix([u])):
    f = sp.Rational(1, 4) * G @ lie(G, V).T - H
    c = lie(H, V)[0] + (u @ u)[0]
    return f, c.simplify().collect([y])

def peq(var, rhss):
    """pretty prints expresions of the kind $A = F(X, Y, Z)$"""
    if not isinstance(var, list):
        var = var.split()
    var_symbs = sp.symbols(" ".join(var))
    if len(var) == 1:
        var_symbs = [var_symbs]
        rhss = [rhss]
    for var_symb, rhs in zip(var_symbs, rhss):
        with sp.evaluate(False):
            display(sp.Eq(var_symb, rhs))

# assumes we're dealing with "concatenated cranks"
def print_problem(V, f, g, c, u=sp.Matrix([u])):
    peq('V', V)
    if not len(f) == 2:
        peq(" ".join([f'\dot{{x}}_{{{i}}} \dot{{y}}_{{{i}}}' for i in range(len(f) // 2)]), f + g @ u)
    else:
        peq("\dot{x} \dot{y}", f + g @ u)
    peq('c', c)

import sympy as sp
import z3
from typing import List, Dict, Tuple, Union, Optional

def sympy_to_z3(expr, variables_map: Dict[str, z3.ArithRef] = None) -> z3.ExprRef:
    """
    Convert a sympy expression to a Z3 expression.

    Args:
        expr: A sympy expression
        variables_map: A dictionary mapping sympy variable names to Z3 variables

    Returns:
        The equivalent Z3 expression
    """
    if variables_map is None:
        variables_map = {}

    # Handle constants
    if expr.is_number:
        return z3.RealVal(float(expr))

    # Handle symbols (variables)
    if expr.is_Symbol:
        var_name = expr.name
        if var_name not in variables_map:
            variables_map[var_name] = z3.Real(var_name)
        return variables_map[var_name]

    # Handle addition
    if expr.is_Add:
        result = z3.RealVal(0)
        for arg in expr.args:
            result = result + sympy_to_z3(arg, variables_map)
        return result

    # Handle multiplication
    if expr.is_Mul:
        result = z3.RealVal(1)
        for arg in expr.args:
            result = result * sympy_to_z3(arg, variables_map)
        return result

    # Handle power
    if expr.is_Pow:
        base = sympy_to_z3(expr.base, variables_map)
        exp = expr.exp

        # Only handle integer exponents for simplicity
        if exp.is_Integer:
            exp_val = int(exp)
            return base ** exp_val
        elif isinstance(exp, sp.Rational):
            return z3.Sqrt(base)
    # if isinstance(expr, sp.cos):
    #     print("*cos*")
    #     return z3.cos(sympy_to_z3(expr.args[0], variables_map))

    # Handle subtraction (will be converted to addition with negative coefficient)

    # Handle division
    if isinstance(expr, sp.Mul) and any(arg.is_Pow and arg.exp.is_negative for arg in expr.args):
        # Extract the numerator and denominator
        num, den = sp.fraction(expr)
        return sympy_to_z3(num, variables_map) / sympy_to_z3(den, variables_map)

    # Add more operations as needed

    raise ValueError(f"Unsupported expression type: {type(expr)}")

def sympy_relation_to_z3(relation, variables_map: Dict[str, z3.ArithRef] = None) -> z3.BoolRef:
    """
    Convert a sympy relation to a Z3 constraint.

    Args:
        relation: A sympy relation (>, >=, =, <, <=, !=)
        variables_map: A dictionary mapping sympy variable names to Z3 variables

    Returns:
        The equivalent Z3 constraint
    """
    if variables_map is None:
        variables_map = {}

    # Check if relation is an actual sympy relational object
    if not isinstance(relation, sp.Rel):
        raise TypeError(f"Expected a sympy relational object, got {type(relation)}. Make sure you're using sympy relations and not Python's built-in comparisons.")

    # Get the left and right hand sides
    lhs, rhs = relation.lhs, relation.rhs

    # Convert to Z3 expressions
    z3_lhs = sympy_to_z3(lhs, variables_map)
    z3_rhs = sympy_to_z3(rhs, variables_map)

    # Create the appropriate Z3 constraint based on the relation type
    if isinstance(relation, sp.Gt):  # >
        return z3_lhs > z3_rhs
    elif isinstance(relation, sp.Ge):  # >=
        return z3_lhs >= z3_rhs
    elif isinstance(relation, sp.Eq):  # =
        return z3_lhs == z3_rhs
    elif isinstance(relation, sp.Lt):  # <
        return z3_lhs < z3_rhs
    elif isinstance(relation, sp.Le):  # <=
        return z3_lhs <= z3_rhs
    elif isinstance(relation, sp.Ne):  # !=
        return z3_lhs != z3_rhs
    else:
        raise ValueError(f"Unsupported relation type: {type(relation)}")

def ensure_sympy_relation(rel):
    """
    Ensure that the input is a sympy relational object.
    If it's already a sympy relation, return it.
    If it's a boolean, raise an error with a helpful message.

    Args:
        rel: A potential sympy relation

    Returns:
        The input if it's a sympy relation

    Raises:
        TypeError: If the input is not a sympy relation
    """
    if isinstance(rel, sp.Rel):
        return rel
    elif isinstance(rel, bool):
        raise TypeError(
            "Received a Python boolean instead of a sympy relation. "
            "This often happens when using '==' with sympy symbols. "
            "Use sp.Eq(x, y) instead of x == y to ensure you get a sympy relational object."
        )
    else:
        raise TypeError(f"Expected a sympy relational object, got {type(rel)}.")

def is_satisfiable(relations: List[sp.Rel], return_counterex=False) -> Tuple[bool, Optional[Dict[str, float]]]:
    """
    Check if a list of sympy relations are simultaneously satisfiable.

    Args:
        relations: A list of sympy relations

    Returns:
        A tuple containing:
            - A boolean indicating if the relations are satisfiable
            - If satisfiable, a dictionary mapping variable names to values
              that satisfy all relations. If not satisfiable, None.
    """
    # Create Z3 solver
    solver = z3.Solver()

    # Map to keep track of variables
    variables_map = {}

    # Convert each relation to Z3 and add to solver
    for relation in relations:
        # Ensure we have a sympy relation
        relation = ensure_sympy_relation(relation)
        z3_constraint = sympy_relation_to_z3(relation, variables_map)
        solver.add(z3_constraint)

    # Check satisfiability
    result = solver.check()
    # print(result)
    if result == z3.sat:
        # Get the model (a satisfying assignment)
        model = solver.model()

        # Convert model to a Python dictionary
        solution = {}
        for var_name, z3_var in variables_map.items():
            if model[z3_var] is not None:
                solution[var_name] = float(model[z3_var].as_decimal(10).replace('?', ''))

        return (True, solution) if return_counterex else True
    else:
        return (False, None) if return_counterex else False


def not_(relations):
    return list(map(lambda s: ~s, relations))


def remove_control(expr): ## assuming SISO
    return expr.subs(u, 0)

def is_pd(F, verbose=False, return_counterex=False):
    F = remove_control(F)
    zero_at_origin = F.subs({key : 0 for key in F.free_symbols}).simplify() == 0
    if not zero_at_origin:
        if verbose:
            print(F, "is not zero at the origin.")
        return (False, None) if return_counterex else False
    v = sp.Matrix(list(F.free_symbols))
    if not v:
        print(F, "has non-positive values everywhere.")
        return (False, None) if return_counterex else False
    has_nonpos_val, counterex = is_satisfiable([(v.T @ v)[0] > 0, F <= 0],
                                                      return_counterex=True)
    if has_nonpos_val and verbose:
        print(F, "has a non-positive value at", counterex)

    return (not has_nonpos_val, counterex) if return_counterex else not has_nonpos_val



def is_psd(F, verbose=False, return_counterex=False):
    F = remove_control(F)
    zero_at_origin = F.subs({key : 0 for key in F.free_symbols}).simplify() == 0
    if not zero_at_origin:
        if verbose:
            print(F, "is not zero at the origin.")
        return (False, None) if return_counterex else False
    has_neg_val, counterex = is_satisfiable([F < 0],
                                            return_counterex=True)
    if has_neg_val and verbose:
        print(F, "has a negative value at", counterex)

    return (not has_neg_val, counterex) if return_counterex else not has_neg_val


def check_pd(V, c):
    V_, c_ = sp.symbols(['V', 'c'])
    res = {}
    for symb, val in zip([V_, c_], [V, c]):
        print("Checking positive-definitness of", symb, "...")
        pd, counterex = is_pd(val, verbose=True, return_counterex=True)
        if pd:
            print("Confirmed.", symb, "is positive-definite.\n")
        else:
            print("Refuted.", symb, "is NOT positive-definite.\n")
        res[str(symb)] = [pd, counterex]
    return res


def pd_feasibility(V, c):
    c = remove_control(c)
    V_z3  = sympy_to_z3(V)
    c_z3  = sympy_to_z3(c)
    return z3.solve([z3.ForAll([z3.Real(x.name), z3.Real(y.name)],
                               z3.Implies(z3.Real(x.name) ** 2 + z3.Real(y.name) ** 2 > 0,
                                          z3.And(V_z3 > 0, c_z3 > 0)))])






# Crank Environments

In [ ]:
## PREPARE FOR RADICAL RENAMING OF PARAMETERS

M, R = sp.symbols('M R')
a, b, d = sp.symbols('a b d')


V = a * x ** 2 + 2 * b * x * y +  d * y ** 2
G = sp.Matrix([0, sp.cos(x)])
H = sp.Matrix([-y,  M*x + R * y + (sp.Rational(1, 4) * G @ G.T @ sp.Matrix([V]).jacobian([x, y]).T)[1]])

print(sp.Matrix([V]).jacobian([x, y]).T)

peq('H', H)

f, c = fc_from_VGH(V, G, H)

print_problem(V, f, G, c)


c_stripped = c.subs(u, 0).subs(sp.cos(x), 0).expand().collect([x, y])
peq("c_{stripped}", c_stripped)

m11 = c_stripped.coeff(x, 2)
m12 = c_stripped.coeff(x, 1).diff(y) / 2
m21 = m12
m22 = c_stripped.coeff(x, 0).diff(y).diff(y) / 2

criteria = [m11 > 0, m11 * m22 - m21 * m12 > 0,
            a > 0, a * d - b ** 2 > 0]

print(criteria)

big_rel = (m11 * m22 - m21 * m12).expand().collect(d)
C = big_rel.coeff(d, 0)
B = big_rel.coeff(d, 1)
A = big_rel.coeff(d, 2)

D = (B**2 - 4 * A * C).simplify()

peq('D', D)

D_root = sp.symbols('D_root')


better_criteria = [M > 0, R > 0, b > 0, a > M * b / R,
                   d > (-B + D_root) / (2 * A),
                   d < (-B - D_root) / (2 * A),
                   sp.Eq(D_root * D_root, D)]
print(better_criteria)

## formal testing
x_z3 = z3.Real(x.name)
y_z3 = z3.Real(y.name)

better_criteria_are_satisfied = z3.And(*list(map(sympy_relation_to_z3, better_criteria)))
but_original_criteria_are_not_satisfied = z3.Not(z3.And(*list(map(sympy_relation_to_z3, criteria))))
print("Improved criteria are satisfiable:", is_satisfiable(better_criteria))
print("It is possible for the original criteria to be unsatisfied:", is_satisfiable(not_(criteria)))
print("It is possible that improved criteria are satisfied while original criteria are not:",
      is_satisfiable(criteria + not_(better_criteria)))

## empirical testing

import numpy as np
from tqdm import tqdm

bottom_d = sp.lambdify([a, b, M, R], (-B + sp.sqrt(D)) / (2 * A))
length_d = sp.lambdify([a, b, M, R], -sp.sqrt(D) / A)

def generate_abdMR(size, range_=10):
    res = np.random.random(size=size * 5).reshape(size, 5) * range_
    for i in range(size):
        _, b, _, M, R = res[i]

        res[i, 0] += M * b / R,
        a = res[i, 0]

        res[i, 2] *= length_d(a, b, M, R) / range_
        res[i, 2] += bottom_d(a, b, M, R)
    return res



num_tests = 100
params = generate_abdMR(num_tests)

count_both_pd = 0
for param in tqdm(params):
    substitution = list(zip([a, b, d, M, R], param))
    c_ = c_stripped.subs(substitution)
    V_ = V.subs(substitution)
    res =  is_pd(c_) and is_pd(V_)
    count_both_pd += res
    if not res:
        print(is_pd(c_, return_counterex=True)[1],
              is_pd(V_, return_counterex=True)[1])
        print(V_, c_, sep=' | ')

print(f"\nRandom tests passed: {count_both_pd}/{num_tests}")



Matrix([[2*a*x + 2*b*y], [2*b*x + 2*d*y]])


Eq(H, Matrix([
[                                     -y],
[M*x + R*y + (2*b*x + 2*d*y)*cos(x)**2/4]]))

[x, y]
[x, y]


Eq(V, a*x**2 + 2*b*x*y + d*y**2)

Eq(\dot{x}, y)

...

Eq(D, 16*M**2*b*(-M*b + R*a))

<ipython-input-16-289fac5ee460>:75: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  res[i, 0] += M * b / R,


[M > 0, R > 0, b > 0, a > M*b/R, d > -(D_root - 2*M*R*b - 2*M*a)/(2*M**2), d < -(-D_root - 2*M*R*b - 2*M*a)/(2*M**2), Eq(D_root**2, 16*M**2*b*(-M*b + R*a))]
Improved criteria are satisfiable: True
It is possible for the original criteria to be unsatisfied: True
It is possible that improved criteria are satisfied while original criteria are not: False


100%|██████████| 100/100 [00:06<00:00, 14.76it/s]


Random tests passed: 100/100




## Integrating cranks



In [ ]:
M, R = 1, 1 # sp.symbols('M R')
a, b, d = 3, 1, 3 # sp.symbols('a b d')
x0, y0, x1, y1 = sp.symbols('x_0 y_0 x_1 y_1')

V_n = a * x ** 2 + 2 * b * x * y +  d * y ** 2
G_n = sp.Matrix([0, sp.cos(x)])
H_n = sp.Matrix([-y,  M*x + R * y + (sp.Rational(1, 4) * G_n @ G_n.T @ sp.Matrix([V_n]).jacobian([x, y]).T)[1]])

u_mult = sp.Matrix(sp.symbols('u_0 u_1'))

subs0 = [(x, x0), (y, y0)]
subs1 = [(x, x1), (y, y1)]


V = V_n.subs(subs0) + V_n.subs(subs1)
G = sp.diag(G_n.subs(subs0), G_n.subs(subs1))
H = sp.Matrix.vstack(H_n.subs(subs0), H_n.subs(subs1))
H[1] += 0.2 * (x0 - x1)
H[3] += 0.2 * (x1 - x0)


print(sp.Matrix([V]).jacobian([x0, y0, x1, y1]).T)

peq('H', H)

f, c = fc_from_VGH(V, G, H)

print_problem(V, f, G, c, u=u_mult)

c_stripped = c.subs(u, 0).subs(sp.cos(x0), 0).subs(sp.cos(x1), 0).expand().collect([x, y])
peq("c_{stripped}", c_stripped)

is_pd(c_stripped)


Matrix([[6*x_0 + 2*y_0], [2*x_0 + 6*y_0], [6*x_1 + 2*y_1], [2*x_1 + 6*y_1]])


Eq(H, Matrix([
[                                                    -y_0],
[ 1.2*x_0 - 0.2*x_1 + y_0 + (2*x_0 + 6*y_0)*cos(x_0)**2/4],
[                                                    -y_1],
[-0.2*x_0 + 1.2*x_1 + y_1 + (2*x_1 + 6*y_1)*cos(x_1)**2/4]]))

[x_0, y_0, x_1, y_1]
[x_0, y_0, x_1, y_1]


Eq(V, 3*x_0**2 + 2*x_0*y_0 + 3*x_1**2 + 2*x_1*y_1 + 3*y_0**2 + 3*y_1**2)

Eq(\dot{x}_{0}, y_0)

...

Eq(\dot{x}_{1}, y_1)

Eq(\dot{y}_{1}, u_1*cos(x_1) + 0.2*x_0 - 1.2*x_1 - y_1)

Eq(c, u**2 - 2*y_0*(3*x_0 + y_0) - 2*y_1*(3*x_1 + y_1) + (x_0 + 3*y_0)*(4.8*x_0 - 0.8*x_1 + 4*y_0 + 2*(x_0 + 3*y_0)*cos(x_0)**2)/2 + (x_1 + 3*y_1)*(-0.8*x_0 + 4.8*x_1 + 4*y_1 + 2*(x_1 + 3*y_1)*cos(x_1)**2)/2)

Eq(c_{stripped}, 2.4*x_0**2 - 0.8*x_0*x_1 + 3.2*x_0*y_0 - 1.2*x_0*y_1 + 2.4*x_1**2 - 1.2*x_1*y_0 + 3.2*x_1*y_1 + 4*y_0**2 + 4*y_1**2)

True

## Generating parameters



The Frobenius norm bounds the operator norm:
https://math.stackexchange.com/questions/252819/why-is-the-frobenius-norm-of-a-matrix-greater-than-or-equal-to-the-spectral-norm .

https://math.hecker.org/2011/06/26/the-inverse-of-a-block-diagonal-matrix/


It is known that a p.d. matrix $A$ stays p.d. after perturbation $\Delta A$ if
$$ \lVert \Delta A \rVert_2\lVert  A^{-1} \rVert_2 < 1.$$

Trivially, this is implied by
$$\lVert \Delta A \rVert_F^2 < \frac{\lvert A \rvert^2}{\lVert{\text{adj }A\rVert_F^2}}$$

Additionally if one assumes diagonal structure of $A$, we obtain:
$$\lVert \Delta A \rVert_F^2 < \frac{\prod_i \lvert A_i \rvert^2}{\sum_j\lVert{\text{adj }A_j\rVert_F^2 }\prod_{i \neq j}\lvert A_i \rvert^2}$$

If one further assumes that $A$ is $2\times 2$ and symmetric, we get
$$\lVert \Delta A \rVert_F^2 < \frac{\prod_i (a_{i11}a_{i22} - a_{i12}^2)^2}{\sum_j (a_{j11}^2 + 2a_{j12}^2 + a_{j22}^2)\prod_{i \neq j}(a_{i11}a_{i22} - a_{i12}^2)^2}$$

or alternatively:
$$\frac{1}{\lVert \Delta A \rVert_F^2} > \sum_j \frac{a_{j11}^2 + 2a_{j12}^2 + a_{j22}^2}{(a_{j11}a_{j22} - a_{j12}^2)^2}$$



In [ ]:
N = 5

def symb_array(names, count=N):
    res = []
    for name in names.split():
        res.append(sp.symbols(" ".join([name + str(i) for i in range(count)])))
    return res

as_, bs, ds, Ms, Rs, Ks = symb_array('a b d M R K')
Ks = Ks[:-1]
xs, ys, us = symb_array('x y u')

state = [s for x_, y_ in zip(xs, ys) for s in (x_, y_)]

M, R = sp.symbols('M R')
a, b, d = sp.symbols('a b d')
K = sp.symbols('K')

V_n = a * x ** 2 + 2 * b * x * y +  d * y ** 2
G_n = sp.Matrix([0, sp.cos(x)])
H_n = sp.Matrix([-y,  M*x + R * y + (sp.Rational(1, 4) * G_n @ G_n.T @ sp.Matrix([V_n]).jacobian([x, y]).T)[1]])




V = \
  sum([V_n.subs([(x, xs[i]), (y, ys[i]), (a, as_[i]), (b, bs[i]), (d, ds[i])])
       for i in range(N)])
G = sp.diag(*[G_n.subs(x, xs[i]) for i in range(N)])
H = sp.Matrix.vstack(*[H_n.subs([(x, xs[i]), (y, ys[i]), (a, as_[i]),
                                 (b, bs[i]), (d, ds[i]), (M, Ms[i]), (R, Rs[i])]) \
                     for i in range(N)])
pad = sp.symbols('pad')
for i in range(N):
    padded_xs = list(xs) + [pad]
    padded_Ks = list(Ks) + [pad]
    H[2 * i + 1] += padded_Ks[i - 1] * (padded_xs[i] - padded_xs[i - 1]) # enacted by previous crank
    H[2 * i + 1] += padded_Ks[i] * (padded_xs[i] - padded_xs[i + 1]) # enacted by next crank
H = H.subs(pad, 0)

# H[1] += 0.1 * (x0 - x1)
# H[3] += 0.1 * (x1 - x0)


print(sp.Matrix([V]).jacobian([x0, y0, x1, y1]).T)

peq('H', H)

f, c = fc_from_VGH(V, G, H, u=sp.Matrix(us))

print_problem(V, f, G, c, u=sp.Matrix(us))

c_stripped = c
for i in range(N):
    c_stripped = c_stripped.subs(us[i], 0).subs(sp.cos(xs[i]), 0)
Q = sp.Matrix([c_stripped]).jacobian(state).T.jacobian(state) / 2

Q_no_springs = Q.subs(list(zip(Ks, [0] * (N - 1))))
DeltaQ = Q - Q_no_springs


peq('c_{stripped|same|K}', c_stripped.subs(list(zip(Ks, [K] * (N - 1)))).expand().collect(K))

peq("\Delta{Q}", DeltaQ)




Matrix([[0], [0], [0], [0]])


Eq(H, Matrix([
[                                                                            -y0],
[                K0*(x0 - x1) + M0*x0 + R0*y0 + (2*b0*x0 + 2*d0*y0)*cos(x0)**2/4],
[                                                                            -y1],
[K0*(-x0 + x1) + K1*(x1 - x2) + M1*x1 + R1*y1 + (2*b1*x1 + 2*d1*y1)*cos(x1)**2/4],
[                                                                            -y2],
[K1*(-x1 + x2) + K2*(x2 - x3) + M2*x2 + R2*y2 + (2*b2*x2 + 2*d2*y2)*cos(x2)**2/4],
[                                                                            -y3],
[K2*(-x2 + x3) + K3*(x3 - x4) + M3*x3 + R3*y3 + (2*b3*x3 + 2*d3*y3)*cos(x3)**2/4],
[                                                                            -y4],
[               K3*(-x3 + x4) + M4*x4 + R4*y4 + (2*b4*x4 + 2*d4*y4)*cos(x4)**2/4]]))

[x0, y0, x1, y1, x2, y2, x3, y3, x4, y4]
[x0, y0, x1, y1, x2, y2, x3, y3, x4, y4]


Eq(V, a0*x0**2 + a1*x1**2 + a2*x2**2 + a3*x3**2 + a4*x4**2 + 2*b0*x0*y0 + 2*b1*x1*y1 + 2*b2*x2*y2 + 2*b3*x3*y3 + 2*b4*x4*y4 + d0*y0**2 + d1*y1**2 + d2*y2**2 + d3*y3**2 + d4*y4**2)

Eq(\dot{x}_{0}, y0)

...

Eq(\dot{x}_{4}, y4)

Eq(\dot{y}_{4}, -K3*(-x3 + x4) - M4*x4 - R4*y4 + u4*cos(x4))

Eq(c, u**2 - 2*y0*(a0*x0 + b0*y0) - 2*y1*(a1*x1 + b1*y1) - 2*y2*(a2*x2 + b2*y2) - 2*y3*(a3*x3 + b3*y3) - 2*y4*(a4*x4 + b4*y4) + (b0*x0 + d0*y0)*(2*K0*(x0 - x1) + 2*M0*x0 + 2*R0*y0 + (b0*x0 + d0*y0)*cos(x0)**2) + (b1*x1 + d1*y1)*(-2*K0*(x0 - x1) + 2*K1*(x1 - x2) + 2*M1*x1 + 2*R1*y1 + (b1*x1 + d1*y1)*cos(x1)**2) + (b2*x2 + d2*y2)*(-2*K1*(x1 - x2) + 2*K2*(x2 - x3) + 2*M2*x2 + 2*R2*y2 + (b2*x2 + d2*y2)*cos(x2)**2) + (b3*x3 + d3*y3)*(-2*K2*(x2 - x3) + 2*K3*(x3 - x4) + 2*M3*x3 + 2*R3*y3 + (b3*x3 + d3*y3)*cos(x3)**2) + (b4*x4 + d4*y4)*(-2*K3*(x3 - x4) + 2*M4*x4 + 2*R4*y4 + (b4*x4 + d4*y4)*cos(x4)**2))

Eq(c_{stripped|same|K}, K*(2*b0*x0**2 - 2*b0*x0*x1 - 2*b1*x0*x1 + 4*b1*x1**2 - 2*b1*x1*x2 - 2*b2*x1*x2 + 4*b2*x2**2 - 2*b2*x2*x3 - 2*b3*x2*x3 + 4*b3*x3**2 - 2*b3*x3*x4 - 2*b4*x3*x4 + 2*b4*x4**2 + 2*d0*x0*y0 - 2*d0*x1*y0 - 2*d1*x0*y1 + 4*d1*x1*y1 - 2*d1*x2*y1 - 2*d2*x1*y2 + 4*d2*x2*y2 - 2*d2*x3*y2 - 2*d3*x2*y3 + 4*d3*x3*y3 - 2*d3*x4*y3 - 2*d4*x3*y4 + 2*d4*x4*y4) + 2*M0*b0*x0**2 + 2*M0*d0*x0*y0 + 2*M1*b1*x1**2 + 2*M1*d1*x1*y1 + 2*M2*b2*x2**2 + 2*M2*d2*x2*y2 + 2*M3*b3*x3**2 + 2*M3*d3*x3*y3 + 2*M4*b4*x4**2 + 2*M4*d4*x4*y4 + 2*R0*b0*x0*y0 + 2*R0*d0*y0**2 + 2*R1*b1*x1*y1 + 2*R1*d1*y1**2 + 2*R2*b2*x2*y2 + 2*R2*d2*y2**2 + 2*R3*b3*x3*y3 + 2*R3*d3*y3**2 + 2*R4*b4*x4*y4 + 2*R4*d4*y4**2 - 2*a0*x0*y0 - 2*a1*x1*y1 - 2*a2*x2*y2 - 2*a3*x3*y3 - 2*a4*x4*y4 - 2*b0*y0**2 - 2*b1*y1**2 - 2*b2*y2**2 - 2*b3*y3**2 - 2*b4*y4**2 + u**2)

Eq(\Delta{Q}, Matrix([
[-2*M0*b0 + b0*(2*K0 + 2*M0), -M0*d0 + d0*(2*K0 + 2*M0)/2,                     -K0*b0 - K0*b1,                             -K0*d1,                                  0,                                  0,                                  0,                                  0,                           0,                           0],
[-M0*d0 + d0*(2*K0 + 2*M0)/2,                           0,                             -K0*d0,                                  0,                                  0,                                  0,                                  0,                                  0,                           0,                           0],
[             -K0*b0 - K0*b1,                      -K0*d0, -2*M1*b1 + b1*(2*K0 + 2*K1 + 2*M1), -M1*d1 + d1*(2*K0 + 2*K1 + 2*M1)/2,                     -K1*b1 - K1*b2,                             -K1*d2,                                  0,                                  0,                           0,      

In [ ]:
DeltaQ_vectorized = DeltaQ.reshape(len(DeltaQ), 1)
DeltaQ_F2 = DeltaQ_vectorized.dot(DeltaQ_vectorized)
peq("||\Delta{Q}||_F^2", DeltaQ_F2.expand().collect(Ks[0]))
P = sp.Matrix([DeltaQ_F2]).jacobian(Ks).T.jacobian(Ks) / 2
peq("P", P)
L, D = ((P + P.T)/2).LDLdecomposition(hermitian=False)
peq("LDL", (L, D))




print("The linear part is zero:")
peq("F", sp.Matrix([DeltaQ_F2]).jacobian(Ks).subs(list(zip(Ks, [0] * (N - 1)))))


# "different Ks can be generated"

## Let's compute the inverse
peq("Q_{initial}", Q_no_springs)
Q_no_springs_inv = sp.diag(*[Q_no_springs[2 * i: 2 * i + 2, 2 * i: 2 * i + 2].inv() for i in range(N)])
peq("Q_{initial|inverse}", Q_no_springs_inv)

Q_no_springs_inv_vec = Q_no_springs_inv.reshape(len(DeltaQ), 1)


rhs = 1 / Q_no_springs_inv_vec.dot(Q_no_springs_inv_vec)


# transform from unit ball that yields valid Ks
transform = L.T.inv() @ D.inv(method="LU").pow(sp.Rational(1, 2)) * sp.sqrt(rhs)


# For same K
K = sp.symbols('K')

peq("||\Delta{Q}||_F^2", DeltaQ_F2.subs(list(zip(Ks, [K] * (N - 1)))).expand().collect(K))

ub_single = sp.sqrt(rhs / (DeltaQ_F2.subs(list(zip(Ks, [K] * (N - 1)))).diff(K).diff(K) / 2))

display(K < ub_single)


Eq(||\Delta{Q}||_F^2, K0**2*(6*b0**2 + 4*b0*b1 + 6*b1**2 + 4*d0**2 + 4*d1**2) + K0*(8*K1*b1**2 + 4*K1*d1**2) + 6*K1**2*b1**2 + 4*K1**2*b1*b2 + 6*K1**2*b2**2 + 4*K1**2*d1**2 + 4*K1**2*d2**2 + 8*K1*K2*b2**2 + 4*K1*K2*d2**2 + 6*K2**2*b2**2 + 4*K2**2*b2*b3 + 6*K2**2*b3**2 + 4*K2**2*d2**2 + 4*K2**2*d3**2 + 8*K2*K3*b3**2 + 4*K2*K3*d3**2 + 6*K3**2*b3**2 + 4*K3**2*b3*b4 + 6*K3**2*b4**2 + 4*K3**2*d3**2 + 4*K3**2*d4**2)

Eq(P, Matrix([
[4*b0**2 + 4*b1**2 + 4*d0**2 + 4*d1**2 + (-2*b0 - 2*b1)*(-b0 - b1),                                                 4*b1**2 + 2*d1**2,                                                                 0,                                                                 0],
[                                                4*b1**2 + 2*d1**2, 4*b1**2 + 4*b2**2 + 4*d1**2 + 4*d2**2 + (-2*b1 - 2*b2)*(-b1 - b2),                                                 4*b2**2 + 2*d2**2,                                                                 0],
[                                                                0,                                                 4*b2**2 + 2*d2**2, 4*b2**2 + 4*b3**2 + 4*d2**2 + 4*d3**2 + (-2*b2 - 2*b3)*(-b2 - b3),                                                 4*b3**2 + 2*d3**2],
[                                                                0,                                                                 0,                                         

Eq(LDL, (Matrix([
[                                                                                      1,                                                                                                                                                                                    0,                                                                                                                                                                                                                                                                                 0, 0],
[(4*b1**2 + 2*d1**2)/(4*b0**2 + 4*b1**2 + 4*d0**2 + 4*d1**2 + (-2*b0 - 2*b1)*(-b0 - b1)),                                                                                                                                                                                    1,                                                                                                                                                                

The linear part is zero:


Eq(F, Matrix([[0, 0, 0, 0]]))

Eq(Q_{initial}, Matrix([
[           2*M0*b0, M0*d0 + R0*b0 - a0,                  0,                  0,                  0,                  0,                  0,                  0,                  0,                  0],
[M0*d0 + R0*b0 - a0,     2*R0*d0 - 2*b0,                  0,                  0,                  0,                  0,                  0,                  0,                  0,                  0],
[                 0,                  0,            2*M1*b1, M1*d1 + R1*b1 - a1,                  0,                  0,                  0,                  0,                  0,                  0],
[                 0,                  0, M1*d1 + R1*b1 - a1,     2*R1*d1 - 2*b1,                  0,                  0,                  0,                  0,                  0,                  0],
[                 0,                  0,                  0,                  0,            2*M2*b2, M2*d2 + R2*b2 - a2,                  0,                  0,       

Eq(Q_{initial|inverse}, Matrix([
[   (-2*R0*d0 + 2*b0)/(M0**2*d0**2 - 2*M0*R0*b0*d0 - 2*M0*a0*d0 + 4*M0*b0**2 + R0**2*b0**2 - 2*R0*a0*b0 + a0**2), (M0*d0 + R0*b0 - a0)/(M0**2*d0**2 - 2*M0*R0*b0*d0 - 2*M0*a0*d0 + 4*M0*b0**2 + R0**2*b0**2 - 2*R0*a0*b0 + a0**2),                                                                                                               0,                                                                                                               0,                                                                                                               0,                                                                                                               0,                                                                                                               0,                                                                                                               0,                                                               

Eq(||\Delta{Q}||_F^2, K**2*(6*b0**2 + 4*b0*b1 + 20*b1**2 + 4*b1*b2 + 20*b2**2 + 4*b2*b3 + 20*b3**2 + 4*b3*b4 + 6*b4**2 + 4*d0**2 + 12*d1**2 + 12*d2**2 + 12*d3**2 + 4*d4**2))

K < sqrt(1/((4*b0**2 + 16*b1**2 + 16*b2**2 + 16*b3**2 + 4*b4**2 + 4*d0**2 + 12*d1**2 + 12*d2**2 + 12*d3**2 + 4*d4**2 + (-2*b0 - 2*b1)*(-b0 - b1) + (-2*b1 - 2*b2)*(-b1 - b2) + (-2*b2 - 2*b3)*(-b2 - b3) + (-2*b3 - 2*b4)*(-b3 - b4))*(4*M0**2*b0**2/(M0**2*d0**2 - 2*M0*R0*b0*d0 - 2*M0*a0*d0 + 4*M0*b0**2 + R0**2*b0**2 - 2*R0*a0*b0 + a0**2)**2 + 4*M1**2*b1**2/(M1**2*d1**2 - 2*M1*R1*b1*d1 - 2*M1*a1*d1 + 4*M1*b1**2 + R1**2*b1**2 - 2*R1*a1*b1 + a1**2)**2 + 4*M2**2*b2**2/(M2**2*d2**2 - 2*M2*R2*b2*d2 - 2*M2*a2*d2 + 4*M2*b2**2 + R2**2*b2**2 - 2*R2*a2*b2 + a2**2)**2 + 4*M3**2*b3**2/(M3**2*d3**2 - 2*M3*R3*b3*d3 - 2*M3*a3*d3 + 4*M3*b3**2 + R3**2*b3**2 - 2*R3*a3*b3 + a3**2)**2 + 4*M4**2*b4**2/(M4**2*d4**2 - 2*M4*R4*b4*d4 - 2*M4*a4*d4 + 4*M4*b4**2 + R4**2*b4**2 - 2*R4*a4*b4 + a4**2)**2 + (-2*R0*d0 + 2*b0)**2/(M0**2*d0**2 - 2*M0*R0*b0*d0 - 2*M0*a0*d0 + 4*M0*b0**2 + R0**2*b0**2 - 2*R0*a0*b0 + a0**2)**2 + (-2*R1*d1 + 2*b1)**2/(M1**2*d1**2 - 2*M1*R1*b1*d1 - 2*M1*a1*d1 + 4*M1*b1**2 + R1**2*b1**2 - 2*R1*a1*b1

In [ ]:
transform_matrix = sp.lambdify(as_ + bs + ds + Ms + Rs, transform)

In [ ]:
def generate_abdMR(size, range_=10):
    res = np.random.random(size=size * 5).reshape(size, 5) * range_
    for i in range(size):
        _, b, _, M, R = res[i]

        res[i, 0] += M * b / R,
        a = res[i, 0]

        res[i, 2] *= length_d(a, b, M, R) / range_
        res[i, 2] += bottom_d(a, b, M, R)
    return res


def sample_unit_ball(dim=N-1):
    while True:
        res = np.random.random(size=dim) * 2 - 1
        if res @ res < 1:
            return res



def gen_params(size, range_=30):
    samples = []
    for i in range(size):
        subs = {}
        params = generate_abdMR(N, range_=range_)
        for i, param in enumerate(params):
            a, b, d, M, R = param
            subs.update({as_[i].name: a, bs[i].name: b, ds[i].name: d, Ms[i].name: M, Rs[i].name: R})
        transform_ = transform_matrix(**subs)
        p = sample_unit_ball()
        subs.update(dict(zip(Ks, (transform_ @ p).tolist())))
        samples.append(subs)
    return samples


num_tests = 1
params = gen_params(num_tests)

count_both_pd = 0
for param in tqdm(params):
    print(param)
    c_ = c_stripped.subs(param)
    V_ = V.subs(param)
    res =  is_pd(c_) and is_pd(V_)
    count_both_pd += res
    if not res:
        print(is_pd(c_, return_counterex=True)[1],
              is_pd(V_, return_counterex=True)[1])
        print(V_, c_, sep=' | ')

print(f"\nRandom tests passed: {count_both_pd}/{num_tests}")






<ipython-input-21-7e01dd0d0ab2>:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  res[i, 0] += M * b / R,
  0%|          | 0/1 [00:00<?, ?it/s]

{'a0': np.float64(27.896982518849462), 'b0': np.float64(7.070674025053045), 'd0': np.float64(7.358649725405502), 'M0': np.float64(15.100027001175018), 'R0': np.float64(24.12888332860114), 'a1': np.float64(17.642332383620246), 'b1': np.float64(29.707684928266247), 'd1': np.float64(72.45956314000165), 'M1': np.float64(11.000592086806435), 'R1': np.float64(26.292491897506125), 'a2': np.float64(10.978509935522624), 'b2': np.float64(13.169952640108031), 'd2': np.float64(121.02285810279565), 'M2': np.float64(1.8072726734359312), 'R2': np.float64(16.291078632037944), 'a3': np.float64(68.21601059163382), 'b3': np.float64(19.544392340467155), 'd3': np.float64(10.643097131241369), 'M3': np.float64(26.71139798575158), 'R3': np.float64(8.231122109011888), 'a4': np.float64(432.4776973337557), 'b4': np.float64(9.122388393100119), 'd4': np.float64(35.225569967758545), 'M4': np.float64(12.420902582842903), 'R4': np.float64(0.26842333483498404), K0: 6.488340927160487e-05, K1: -7.3107370920321445e-06, K

100%|██████████| 1/1 [01:41<00:00, 101.74s/it]


Random tests passed: 1/1
